In [1]:
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
img_arr = []
for path_name in glob.glob('../Waifu-GAN/?*'):
    img = Image.open(path_name)
    img_arr.append(img)

In [3]:
train, test = train_test_split(img_arr, test_size=0.3, random_state=99)
train, val = train_test_split(train, test_size=0.3, random_state=99)

In [4]:
train_size = len(train)
test_size = len(test)
val_size = len(val)

print(train_size, test_size, val_size)

44 28 20


In [5]:
rotation_range = 15
width_shift_range = 0.125
height_shift_range = 0.125
shear_range = 0.1
zoom_range = 0.125
horizontal_flip = True

noise_mean = 60
noise_std = 60
noise_factor = 0.75

In [6]:
def add_gaussian_noise(img):

    img_noisy = img+(noise_factor * (np.random.normal(loc=noise_mean,scale=noise_std,size=img.shape)))
    img_noisy = np.clip(img_noisy, 0, 255)
    return img_noisy

In [7]:
datagen = ImageDataGenerator( rotation_range=rotation_range,
width_shift_range=width_shift_range,
height_shift_range=height_shift_range,
shear_range=shear_range,
zoom_range=zoom_range,
horizontal_flip=horizontal_flip,
fill_mode = 'constant',
preprocessing_function = add_gaussian_noise)

In [8]:
# Prepare train data
count = 0
for img in train:
    width, height = img.size
    resized_img = img.resize((320, 320), resample=Image.LANCZOS)
    small_img = resized_img.resize((80, 80), resample=Image.LANCZOS)
    resized_img = np.array(resized_img, ndmin=4)
    small_img = np.array(small_img, ndmin=4)
    for _ in range(10):
        flow = datagen.flow(small_img, resized_img, batch_size=1)
        gen_img = flow.next()
        cv2.imwrite('datasets/hq_train/tr' + str(count) + '.png', cv2.cvtColor(gen_img[1][0], cv2.COLOR_RGB2BGR))
        cv2.imwrite('datasets/lq_train/tr' + str(count) + '.png', cv2.cvtColor(gen_img[0][0], cv2.COLOR_RGB2BGR))
        count += 1

In [9]:
# Prepare validation data
count = 0
for img in val:
    width, height = img.size
    resized_img = img.resize((320, 320), resample=Image.LANCZOS)
    small_img = resized_img.resize((80, 80), resample=Image.LANCZOS)
    resized_img = np.array(resized_img, ndmin=4)
    small_img = np.array(small_img, ndmin=4)
    for _ in range(10):
        flow = datagen.flow(small_img, resized_img, batch_size=1)
        gen_img = flow.next()
        cv2.imwrite('datasets/hq_val/val' + str(count) + '.png', cv2.cvtColor(gen_img[1][0], cv2.COLOR_RGB2BGR))
        cv2.imwrite('datasets/lq_val/val' + str(count) + '.png', cv2.cvtColor(gen_img[0][0], cv2.COLOR_RGB2BGR))
        count += 1

In [10]:
# Prepare test data
count = 0
for img in test:
    width, height = img.size
    resized_img = img.resize((320, 320), resample=Image.LANCZOS)
    small_img = resized_img.resize((80, 80), resample=Image.LANCZOS)
    resized_img = np.array(resized_img, ndmin=4)
    small_img = np.array(small_img, ndmin=4)
    for _ in range(10):
        flow = datagen.flow(small_img, resized_img, batch_size=1)
        gen_img = flow.next()
        cv2.imwrite('datasets/hq_test/te' + str(count) + '.png', cv2.cvtColor(gen_img[1][0], cv2.COLOR_RGB2BGR))
        cv2.imwrite('datasets/lq_test/te' + str(count) + '.png', cv2.cvtColor(gen_img[0][0], cv2.COLOR_RGB2BGR))
        count += 1